# Run an experiment script
In the previous example, we ran a experiment inline in a notebook. 
A more flexible solution is to create a separate script for the experiment, and store it in a folder along with any other files it needs, and then use Azure ML to run the experiment based on the script in the folder.

In [ ]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

for compute_name in ws.compute_targets:
    compute = ws.compute_targets[compute_name]
    print("(",compute.status.state ,") ",compute.name, ":", compute.type)

To run the script, we must create a **ScriptRunConfig** that identifies the Python script file to be run in the experiment, and then run an experiment based on it.

> **Note**: The ScriptRunConfig also determines the compute target and Python environment. In this case, the Python environment is defined to include some Conda and pip packages, but the compute target is omitted; so the default local compute will be used.

The following cell configures and submits the script-based experiment.

In [ ]:
from azureml.core import Experiment, ScriptRunConfig, Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.widgets import RunDetails

# Create a Python environment for the experiment
experiment_env = Environment("diabetes-classification")

# Ensure the required packages are installed
packages = CondaDependencies.create(conda_packages=['scikit-learn','pip'],
                                    pip_packages=['azureml-defaults'])
experiment_env.python.conda_dependencies = packages

## Running on Local Machine

In [ ]:
experiment_env.python.user_managed_dependencies = True
computeTarget = "local" # Default

## Running on a Compute Instance in Azure

In [ ]:
experiment_env.python.user_managed_dependencies = False # Default Value
computeTarget = ws.compute_targets["basic-instance"]#Environment.get(workspace=ws, name='basic-instance')

## Create a script config

In [ ]:
script_config = ScriptRunConfig(source_directory='../scripts',
                                script='diabetes_logReg.py',
                                arguments = ['--reg-rate', 1], # Passing Command Line Arguments
                                compute_target = computeTarget,  # Assigning Compute Targets. Default = Local Machine
                                environment=experiment_env) 

## Submit the experiment
The experiment trains Asynchronously

In [ ]:
experiment = Experiment(workspace=ws, name='diabetes-classification-experiment')
run = experiment.submit(config=script_config)

RunDetails(run).show()

## Wait for Run Completed

In [ ]:
run.wait_for_completion()

## Current Run Files

In [ ]:
# Get output files
print("\nFiles:")
files = run.get_file_names()
for file in files:
    print(file)

## Run History

In [ ]:
from azureml.core import Experiment, Run

diabetes_experiment = ws.experiments['diabetes-classification-experiment']
for logged_run in diabetes_experiment.get_runs():
    print('Run ID:', logged_run.id)
    metrics = logged_run.get_metrics()
    for key in metrics.keys():
        print('-', key, metrics.get(key))